In [6]:
import numpy as np
from scipy.optimize import minimize

def estimate_distortion_params(distorted_images, undistorted_images, K_undistorted):
    # Assuming distortion parameters to be estimated (k1, k2, p1, p2, k3)
    initial_params = np.zeros(5)  # Initial guess for distortion parameters (k1, k2, p1, p2, k3)

    # Define the objective function to minimize
    def objective_function(params):
        k1, k2, p1, p2, k3 = params

        # Initialize total error
        total_error = 0

        for distorted_img, undistorted_img in zip(distorted_images, undistorted_images):
            # Undistorted points based on the intrinsic matrix K_undistorted
            h, w = distorted_img.shape[:2]
            grid_x, grid_y = np.meshgrid(np.arange(w), np.arange(h))
            points = np.stack([grid_x, grid_y], axis=-1).reshape(-1, 2)

            # Apply distortion model
            r2 = np.sum(points ** 2, axis=-1)
            r4 = r2 ** 2
            radial_distortion = 1 + k1 * r2 + k2 * r4 + k3 * r4 * r2
            tangential_distortion_x = 2 * p1 * points[:, 0] * points[:, 1] + p2 * (r2 + 2 * points[:, 0] ** 2)
            tangential_distortion_y = p1 * (r2 + 2 * points[:, 1] ** 2) + 2 * p2 * points[:, 0] * points[:, 1]

            # Calculate the distorted points by applying radial and tangential distortion
            distorted_points = points * radial_distortion[:, None] + np.stack([tangential_distortion_x, tangential_distortion_y], axis=-1)

            # Reshape distorted points back to image grid
            distorted_points = distorted_points.reshape(h, w, 2)

            # Convert distorted image to grayscale
            distorted_img_grayscale = distorted_img.mean(axis=-1)

            # Calculate the error between projected points and distorted image points
            u, v = distorted_points[..., 0], distorted_points[..., 1]

            # Ensure u and v are within image bounds
            u = np.clip(u, 0, w - 1)
            v = np.clip(v, 0, h - 1)

            # Create an error map based on the grayscale values at the projected coordinates
            error_map = distorted_img_grayscale[v.astype(int), u.astype(int)]

            # Sum squared differences
            total_error += np.sum((error_map - distorted_img_grayscale) ** 2)

        return float(total_error)  # Ensure the error is a scalar

    # Perform optimization
    result = minimize(objective_function, initial_params, method='L-BFGS-B')
    estimated_params = result.x

    return estimated_params


# Example usage:

# Example distorted and undistorted image data (replace with actual images)
distorted_images = [np.random.rand(480, 640, 3) for _ in range(5)]  # List of distorted images
undistorted_images = [np.random.rand(480, 640, 3) for _ in range(5)]  # List of undistorted images

# Example camera intrinsic matrix (K_undistorted)
K_undistorted = np.array([[1000, 0, 500], [0, 1000, 500], [0, 0, 1]])

# Estimate distortion parameters
estimated_params = estimate_distortion_params(distorted_images, undistorted_images, K_undistorted)
print("Estimated distortion parameters:", estimated_params)


Estimated distortion parameters: [0. 0. 0. 0. 0.]
